In [2]:
from rca import Db, Spotify
import pandas as pd
import numpy as np
import statistics as stats
import datetime as datetime
from datetime import timedelta

# Release Date Function (just for predicting release date)

In [26]:
def cleanReleaseDate(df):
    """
    Looks at the streaming data for a song surrounding the release date provided by Nielsen
    and predicts what the actual release date of that song is based on the largest day-to-day
    streaming increase within 21 days of the first recorded non-zero streaming day
    ----
    Also works with a df having than 1 song
    """
    
    # Preliminary cleaning
    df.fillna(0, inplace=True)
    df['streams'] = df['streams'].astype(int)
    df['date'] = pd.to_datetime(df['date'])
    df['release_date'] = pd.to_datetime(df['release_date'])
    df['artist'] = df['artist'].astype(str)
    
    # Creating a day-to-day streams difference column
    df_copy = df.copy()
    df_copy['streams_diff'] = df_copy['streams'].diff()
    
    # Initialize list to store release dates
    all_release_dates = []
    
    # Process each song group to find release dates
    for song_id in df_copy['song_id'].unique():
        song_group = df_copy[df_copy['song_id'] == song_id]
        
        # Find first non-zero streams date
        first_streams = song_group[song_group['streams'] > 0]['date'].min()
        if pd.isnull(first_streams):
            continue
            
        # Create window of 21 days from first streams
        window = song_group[
            (song_group['date'] >= first_streams) & 
            (song_group['date'] <= first_streams + pd.Timedelta(days=21))
        ]
        
        # Get index of maximum stream difference
        release_idx = window['streams_diff'].idxmax()
        all_release_dates.append(release_idx)
    
    # Get full rows for release dates
    release_dates = df_copy.loc[all_release_dates]
    
    # Initialize list to store post-release dataframes
    post_releases = []
    
    # Process each song's release date
    for song_id, release_row in release_dates.iterrows():
        song_data = df[df['song_id'] == release_row['song_id']]
        release_date = release_row['date']
        
        # Get data from release date forward
        post_release = song_data[song_data['date'] >= release_date].copy()
        post_release.loc[:, 'release_date'] = release_date
        post_releases.append(post_release)
    
    # Single concat at the end
    return pd.concat(post_releases, ignore_index=True)

In [27]:
country_df = pd.read_csv('country.csv')
country_df

,song_id,artist,title,release_date,date,streams
0,1531675044,Grant Gilbert,Turn It Down,2023-08-25,2023-08-13,0.0
1,1531675044,Grant Gilbert,Turn It Down,2023-08-25,2023-08-14,0.0
2,1531675044,Grant Gilbert,Turn It Down,2023-08-25,2023-08-15,0.0
3,1531675044,Grant Gilbert,Turn It Down,2023-08-25,2023-08-16,0.0
4,1531675044,Grant Gilbert,Turn It Down,2023-08-25,2023-08-17,0.0
...,...,...,...,...,...,...
664397,995180282,Graham Barham,Beer By My Bed,2023-02-10,2024-11-26,7889.0
664398,995180282,Graham Barham,Beer By My Bed,2023-02-10,2024-11-27,7857.0
664399,995180282,Graham Barham,Beer By My Bed,2023-02-10,2024-11-28,5784.0
664400,995180282,Graham Barham,Beer By My Bed,2023-02-10,2024-11-29,5988.0


In [28]:
clean_country = cleanReleaseDate(country_df)
clean_country

,song_id,artist,title,release_date,date,streams
0,1531675044,Grant Gilbert,Turn It Down,2023-08-25,2023-08-25,15472
1,1531675044,Grant Gilbert,Turn It Down,2023-08-25,2023-08-26,10151
2,1531675044,Grant Gilbert,Turn It Down,2023-08-25,2023-08-27,8042
3,1531675044,Grant Gilbert,Turn It Down,2023-08-25,2023-08-28,10159
4,1531675044,Grant Gilbert,Turn It Down,2023-08-25,2023-08-29,10122
...,...,...,...,...,...,...
647239,995180282,Graham Barham,Beer By My Bed,2023-02-10,2024-11-26,7889
647240,995180282,Graham Barham,Beer By My Bed,2023-02-10,2024-11-27,7857
647241,995180282,Graham Barham,Beer By My Bed,2023-02-10,2024-11-28,5784
647242,995180282,Graham Barham,Beer By My Bed,2023-02-10,2024-11-29,5988


In [29]:
print(len(country_df['song_id'].unique()))
print(len(clean_country['song_id'].unique()))

1295
1295
